In [1]:
from tqdm import tqdm

In [2]:
# import and pre-load funtion
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *
# from google.colab import files
# uploaded = files.upload()
# %run func.py

In [4]:
home_rob_original = get_datataipei_api('93d9bc2d-af08-4db7-a56b-9f0a49226fa3')
motorcycle_rob_original = get_datataipei_api('ac508aeb-9f26-409c-9fb0-20c65a973498')
car_rob_original = get_datataipei_api('967faed7-ea9b-4698-970a-d335d5e4ccc3')
bike_rob_original = get_datataipei_api('adf80a2b-b29d-4fca-888c-bcd26ae314e0')
street_rob_original = get_datataipei_api('6ecb4c41-fbc9-4b04-b182-a7da6c780f8d')
street_theft_original = get_datataipei_api('37b75aab-e504-413a-911d-987651aacbfb')
print(home_rob_original)

       _id                                        _importdate    編號    案類  \
0        1  {'date': '2023-11-15 09:54:17.073642', 'timezo...     1  住宅竊盜   
1        2  {'date': '2023-11-15 09:54:17.085777', 'timezo...     2  住宅竊盜   
2        3  {'date': '2023-11-15 09:54:17.087297', 'timezo...     3  住宅竊盜   
3        4  {'date': '2023-11-15 09:54:17.089023', 'timezo...     4  住宅竊盜   
4        5  {'date': '2023-11-15 09:54:17.090595', 'timezo...     5  住宅竊盜   
...    ...                                                ...   ...   ...   
3714  3715  {'date': '2023-11-15 09:54:24.847957', 'timezo...  3715  住宅竊盜   
3715  3716  {'date': '2023-11-15 09:54:24.850920', 'timezo...  3716  住宅竊盜   
3716  3717  {'date': '2023-11-15 09:54:24.852724', 'timezo...  3717  住宅竊盜   
3717  3718  {'date': '2023-11-15 09:54:24.856667', 'timezo...  3718  住宅竊盜   
3718  3719  {'date': '2023-11-15 09:54:24.858479', 'timezo...  3719  住宅竊盜   

         發生日期   發生時段                        發生地點  
0     1030623  08~10    

In [4]:
home_rob_copy = home_rob_original.copy()
motorcycle_rob_copy = motorcycle_rob_original.copy()
car_rob_copy = car_rob_original.copy()
bike_rob_copy = bike_rob_original.copy()
street_rob_copy = street_rob_original.copy()
street_theft_copy = street_theft_original.copy()

In [5]:
print(home_rob_copy)
print(motorcycle_rob_copy)
print(car_rob_copy)
print(bike_rob_copy)
print(street_rob_copy)

       _id                                        _importdate    編號    案類  \
0        1  {'date': '2023-11-15 09:54:17.073642', 'timezo...     1  住宅竊盜   
1        2  {'date': '2023-11-15 09:54:17.085777', 'timezo...     2  住宅竊盜   
2        3  {'date': '2023-11-15 09:54:17.087297', 'timezo...     3  住宅竊盜   
3        4  {'date': '2023-11-15 09:54:17.089023', 'timezo...     4  住宅竊盜   
4        5  {'date': '2023-11-15 09:54:17.090595', 'timezo...     5  住宅竊盜   
...    ...                                                ...   ...   ...   
3714  3715  {'date': '2023-11-15 09:54:24.847957', 'timezo...  3715  住宅竊盜   
3715  3716  {'date': '2023-11-15 09:54:24.850920', 'timezo...  3716  住宅竊盜   
3716  3717  {'date': '2023-11-15 09:54:24.852724', 'timezo...  3717  住宅竊盜   
3717  3718  {'date': '2023-11-15 09:54:24.856667', 'timezo...  3718  住宅竊盜   
3718  3719  {'date': '2023-11-15 09:54:24.858479', 'timezo...  3719  住宅竊盜   

         發生日期   發生時段                        發生地點  
0     1030623  08~10    

In [6]:
car_rob_copy = car_rob_copy[car_rob_copy["發生地點"].str.len()>0].reset_index(drop=True)

In [7]:
available_year = ('107', '108', '109', '110', '111')
home_rob_date = home_rob_copy["發生日期"].str[:-4]

motorcycle_rob_date = motorcycle_rob_copy["發生日期"].str[:-4]
car_rob_date = car_rob_copy["發生日期"].str[:-4]
bike_rob_date = bike_rob_copy["發生日期"].str[:-4]
street_rob_date = street_rob_copy["發生日期"].str[:-4]
street_theft_date = street_theft_copy["發生日期"].str[:-4]
data_list = [home_rob_date, motorcycle_rob_date, car_rob_date, bike_rob_date, street_rob_date, street_theft_date]
for i in range(5):
  data_list[i] = data_list[i][data_list[i].str.len()==3]
  data_list[i] = data_list[i][data_list[i].str.endswith(available_year)].reset_index(drop=True)
  # print(data_list[i])
[home_rob_date, motorcycle_rob_date, car_rob_date, bike_rob_date, street_rob_date, street_theft_date] = data_list
home_rob_date.name="住宅竊盜"
motorcycle_rob_date.name="機車竊盜"
car_rob_date.name="汽車竊盜"
bike_rob_date.name="自行車竊盜"
street_rob_date.name="街頭搶奪"
street_theft_date.name="街頭強盜"
home_rob_date

0       107
1       107
2       107
3       107
4       107
       ... 
1630    111
1631    111
1632    111
1633    108
1634    111
Name: 住宅竊盜, Length: 1635, dtype: object

In [8]:
years_list = []
for i in range(107, 112):
  years_list.append(str(i))
years_list

['107', '108', '109', '110', '111']

In [9]:
home_rob_dict = dict.fromkeys(years_list, 0)
motorcycle_rob_dict = dict.fromkeys(years_list, 0)
car_rob_dict = dict.fromkeys(years_list, 0)
bike_rob_dict = dict.fromkeys(years_list, 0)
street_rob_dict = dict.fromkeys(years_list, 0)
street_theft_dict = dict.fromkeys(years_list, 0)
dict_list = [home_rob_dict, motorcycle_rob_dict, car_rob_dict, bike_rob_dict, street_rob_dict, street_theft_dict]
data_list = [home_rob_date, motorcycle_rob_date, car_rob_date, bike_rob_date, street_rob_date, street_theft_date]
for i in range(len(data_list)):
  for j in range(len(data_list[i])):
    if(data_list[i][j] in dict_list[i]):
      dict_list[i][data_list[i][j]] += 1
for i in range(6):
  dict_list[i]["data"]=data_list[i]

In [10]:
home_rob_data_output = home_rob_dict.items()
results = {
    'data': [
        {
            'name': col['data'].name,
            'data': [
                {
                    'x': x,
                    'y': y
                }
                for x, y in list(filter(lambda x: x[0]!="data", col.items()))
            ]
        }
        for col in dict_list
    ]
}
results


{'data': [{'name': '住宅竊盜',
   'data': [{'x': '107', 'y': 502},
    {'x': '108', 'y': 413},
    {'x': '109', 'y': 307},
    {'x': '110', 'y': 202},
    {'x': '111', 'y': 211}]},
  {'name': '機車竊盜',
   'data': [{'x': '107', 'y': 778},
    {'x': '108', 'y': 378},
    {'x': '109', 'y': 291},
    {'x': '110', 'y': 290},
    {'x': '111', 'y': 248}]},
  {'name': '汽車竊盜',
   'data': [{'x': '107', 'y': 98},
    {'x': '108', 'y': 58},
    {'x': '109', 'y': 35},
    {'x': '110', 'y': 30},
    {'x': '111', 'y': 33}]},
  {'name': '自行車竊盜',
   'data': [{'x': '107', 'y': 312},
    {'x': '108', 'y': 355},
    {'x': '109', 'y': 292},
    {'x': '110', 'y': 384},
    {'x': '111', 'y': 295}]},
  {'name': '街頭搶奪',
   'data': [{'x': '107', 'y': 9},
    {'x': '108', 'y': 9},
    {'x': '109', 'y': 9},
    {'x': '110', 'y': 5},
    {'x': '111', 'y': 4}]},
  {'name': '街頭強盜',
   'data': [{'x': '107', 'y': 5},
    {'x': '108', 'y': 7},
    {'x': '109', 'y': 6},
    {'x': '110', 'y': 2},
    {'x': '111', 'y': 2}]}]}

In [11]:
with open('3001.json', "w") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

In [12]:
home_rob_date_address = home_rob_copy[["發生日期", "發生地點"]]
motorcycle_rob_date_address = motorcycle_rob_copy[["發生日期", "發生地點"]]
car_rob_date_address = car_rob_copy[["發生日期", "發生地點"]]
bike_rob_date_address = bike_rob_copy[["發生日期", "發生地點"]]
street_rob_date_address = street_rob_copy[["發生日期", "發生地點"]]
street_theft_date_address = street_theft_copy[["發生日期", "發生地點"]]
df_list = [home_rob_date_address, motorcycle_rob_date_address, car_rob_date_address, bike_rob_date_address, street_rob_date_address, street_theft_date_address]
for i in range(len(df_list)):
  # df_list[i]['geometry'] = df_list[i]['發生地點'].apply(
  #   lambda x: Point(
  #       geocoder.arcgis(x).json['lng'],
  #       geocoder.arcgis(x).json['lat']
  #   )
  # )
  df_list[i]["發生日期"] = df_list[i]["發生日期"].apply(
    lambda x: x[:-4]
  )
  df_list[i] = df_list[i][df_list[i]["發生日期"].str.len()==3]
  df_list[i] = df_list[i][df_list[i]["發生日期"].str.endswith(available_year)].reset_index(drop=True)
  # print(data_list[i])
[home_rob_date_address, motorcycle_rob_date_address, car_rob_date_address, bike_rob_date_address, street_rob_date_address, street_theft_date_address] = df_list
home_rob_date_address.name="住宅竊盜"
motorcycle_rob_date_address.name="機車竊盜"
car_rob_date_address.name="汽車竊盜"
bike_rob_date_address.name="自行車竊盜"
street_rob_date_address.name="街頭搶奪"
street_theft_date_address.name="街頭強盜"
home_rob_date_address

C:\Users\10413\AppData\Local\Temp\ipykernel_3928\3040918163.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[i]["發生日期"] = df_list[i]["發生日期"].apply(
C:\Users\10413\AppData\Local\Temp\ipykernel_3928\3040918163.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[i]["發生日期"] = df_list[i]["發生日期"].apply(
C:\Users\10413\AppData\Local\Temp\ipykernel_3928\3040918163.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

,發生日期,發生地點
0,107,臺北市中山區國中路151~180號
1,107,臺北市文山區興隆路2段78巷4弄1~30號
2,107,臺北市中山區中山北路2段77巷1~30號
3,107,臺北市中正區林興里水源路25巷1~30號
4,107,臺北市中正區新生南路1段1~30號
...,...,...
1630,111,臺北市北投區建民里文林北路80巷38弄1~30號
1631,111,臺北市信義區泰和里吳興街600巷76弄121~150號
1632,111,臺北市萬華區長沙街2段61~90號
1633,108,臺北市大同區南京西路64巷20弄1~30號


In [9]:
def getCoordinate(addr):
  addr_tmp = addr
  if(addr[:3]!="台北市" or addr[:3]!="臺北市"):
    addr_tmp = "台北市" + addr
  if("鄰" in addr and len(addr)>20):
    idx = addr.index('鄰')
    addr_tmp = "台北市"+addr_tmp[idx+1:]

  if("里" in addr and len(addr)>20):
    idx = addr.index('里')
    addr_tmp = "台北市"+addr_tmp[idx+1:]
  addr_info = geocoder.mapbox(addr_tmp, key="pk.eyJ1IjoiY2hlbmpvYWNoaW0iLCJhIjoiY2xva3dneWRyMGYwdjJxbnJzb3g4NWEycSJ9.MYCldpZlb8AOmIG-cyBVsg") # address to coordinate
  addr_json = addr_info.json
  # print(addr_json)
  try:
    if(addr_json != None):
      # print(addr_json['lng'])
      return Point(
        addr_json['lng'],
        addr_json['lat']
      )
    addr_info = geocoder.arcgis(addr_tmp) # address to coordinate
    addr_json = addr_info.json
    if(addr_json != None):
      # print(addr_json['lng'])
      return Point(
        addr_json['lng'],
        addr_json['lat']
      )
    return 'NA'
  except:
    return 'NA'
# g=geocoder.google("Mountain View, CA")
# print(g.json)
# print()
# street_theft_date_address['geometry'] = street_theft_date_address['發生地點'].apply(
#   lambda x: getCoordinate(x)
# )
# street_theft_date_address

In [67]:
street_theft_date_address['geometry'] = street_theft_date_address['發生地點'].apply(
  lambda x: getCoordinate(x)
)
street_theft_date_address

,發生日期,發生地點,geometry
0,107,台北市中山區中山北路2段31~60號,POINT (121.523408 25.070766)
1,107,台北市北投區福興里致遠一路2段1~30號,POINT (121.5137848 25.1120554)
2,107,台北市士林區舊佳里中正路211~240號,POINT (121.525669 25.097521)
3,107,台北市大安區永康里永康街1~30號,POINT (121.529768 25.033221)
4,107,台北市文山區博嘉里020鄰木柵路4段1~30號,POINT (121.5529323 25.0173729)
5,108,台北市文山區景行里車前路29巷,POINT (121.5404525 24.9911322)
6,108,台北市信義區三犁里象山登山步道(煙火觀景臺),POINT (121.570528 25.027865)
7,108,台北市北投區湖田里陽金公路(馬槽橋下方),POINT (121.558576 25.184569)
8,108,台北市信義區虎林街181~210號,POINT (121.576835 25.040163)
9,108,台北市信義區永吉路225巷4弄1~30號,POINT (121.5736065 25.0461391)


In [68]:
for i in tqdm(range(len(df_list))):
  df_list[i]['geometry'] = df_list[i]['發生地點'].apply(
    lambda x: getCoordinate(x)
  )
  df_list[i] = df_list[i][df_list[i]['geometry']!='NA'].reset_index(drop=True)
[home_rob_date_address, motorcycle_rob_date_address, car_rob_date_address, bike_rob_date_address, street_rob_date_address, street_theft_date_address] = df_list
home_rob_date_address

  0%|          | 0/6 [00:00<?, ?it/s]Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/臺北市北投區奇岩里公?路91~120號.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/%E8%87%BA%E5%8C%97%E5%B8%82%E5%8C%97%E6%8A%95%E5%8D%80%E5%A5%87%E5%B2%A9%E9%87%8C%E5%85%AC?%E8%B7%AF91~120%E8%99%9F.json&access_token=pk.eyJ1IjoiY2hlbmpvYWNoaW0iLCJhIjoiY2xva3dneWRyMGYwdjJxbnJzb3g4NWEycSJ9.MYCldpZlb8AOmIG-cyBVsg
Status code 422 from https://api.mapbox.com/geocoding/v5/mapbox.places/台北市承德路4段2巷(士林區承德路四段2巷上).json: ERROR - 422 Client Error: Unknown for url: https://api.mapbox.com/geocoding/v5/mapbox.places/%E5%8F%B0%E5%8C%97%E5%B8%82%E6%89%BF%E5%BE%B7%E8%B7%AF4%E6%AE%B52%E5%B7%B7(%E5%A3%AB%E6%9E%97%E5%8D%80%E6%89%BF%E5%BE%B7%E8%B7%AF%E5%9B%9B%E6%AE%B52%E5%B7%B7%E4%B8%8A).json?access_token=pk.eyJ1IjoiY2hlbmpvYWNoaW0iLCJhIjoiY2xva3dneWRyMGYwdjJxbnJzb3g4NWEycSJ9.MYCldpZlb8AOmIG-cyBVsg
Status code 422 from https://api.mapbox.com/geocoding/v5/mapbox.pla

,發生日期,發生地點,geometry
0,107,臺北市中山區國中路151~180號,POINT (121.533468 25.064361)
1,107,臺北市文山區興隆路2段78巷4弄1~30號,POINT (121.555296 25.000257)
2,107,臺北市中山區中山北路2段77巷1~30號,POINT (121.523408 25.070766)
3,107,臺北市中正區林興里水源路25巷1~30號,POINT (121.5254013 25.0174387)
4,107,臺北市中正區新生南路1段1~30號,POINT (121.532775 25.042293)
...,...,...,...
1630,111,臺北市北投區建民里文林北路80巷38弄1~30號,POINT (121.5194692 25.1063238)
1631,111,臺北市信義區泰和里吳興街600巷76弄121~150號,POINT (121.5699722 25.0181758)
1632,111,臺北市萬華區長沙街2段61~90號,POINT (121.5040773 25.0403531)
1633,108,臺北市大同區南京西路64巷20弄1~30號,POINT (121.51825729999999 25.05149835)


In [83]:
home_rob_date_address.to_pickle("home_rob_date_address.pkl")
motorcycle_rob_date_address.to_pickle("motorcycle_rob_date_address.pkl")
car_rob_date_address.to_pickle("car_rob_date_address.pkl")
bike_rob_date_address.to_pickle("bike_rob_date_address.pkl")
street_rob_date_address.to_pickle("street_rob_date_address.pkl")
street_theft_date_address.to_pickle("street_theft_date_address.pkl")

In [8]:
ob_gyn_original = get_datataipei_api('ba860191-ad03-49a3-8df4-29e0a01f3026')
ob_gyn_copy = ob_gyn_original.copy()
ob_gyn_address = ob_gyn_copy[["機構名稱"]]
ob_gyn_address

,機構名稱
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所
1,祈新婦產科診所
2,江恩婦產科診所
3,小禾馨懷寧小兒專科診所
4,國立台灣大學醫學院附設醫院
...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理
139,愛文診所
140,仁荔健康診所
141,愛文婦產科診所


In [12]:
def getCoordinateOfHospital(addr):
  addr_tmp = addr
  if(addr[:3]!="台北市" or addr[:3]!="臺北市"):
    addr_tmp = "台北市" + addr
  if("醫院" in addr_tmp):
    addr_tmp = addr_tmp[:addr_tmp.index('院')+1]
  addr_info = geocoder.arcgis(addr_tmp) # address to coordinate
  addr_json = addr_info.json
  # print(addr_json)
  try:
    if(addr_json != None):
      # print(addr_json['lng'])
      return Point(
        addr_json['lng'],
        addr_json['lat']
      )
    return 'NA'
  except:
    return 'NA'

In [13]:
ob_gyn_address['geometry'] = ob_gyn_address['機構名稱'].apply(
    lambda x: getCoordinateOfHospital(x)
)
ob_gyn_address

C:\Users\10413\AppData\Local\Temp\ipykernel_20368\1842139510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_gyn_address['geometry'] = ob_gyn_address['機構名稱'].apply(


,機構名稱,geometry
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,POINT (121.51093000000003 25.04302000000007)
1,祈新婦產科診所,POINT (121.56355000000008 25.037370000000067)
2,江恩婦產科診所,POINT (121.51695000000007 25.045440000000042)
3,小禾馨懷寧小兒專科診所,POINT (121.51333000000011 25.043040000000076)
4,國立台灣大學醫學院附設醫院,POINT (121.53369000000009 25.01697000000007)
...,...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,POINT (121.55767000000003 24.999630000000025)
139,愛文診所,POINT (121.55653000000007 25.00022000000007)
140,仁荔健康診所,POINT (121.54491000000007 24.998620000000074)
141,愛文婦產科診所,POINT (121.56355000000008 25.037370000000067)


In [33]:
ob_gyn_address.to_pickle("ob_gyn_address.pkl")

In [7]:
ob_gyn_address=pd.read_pickle("ob_gyn_address.pkl")
ob_gyn_address.loc[[5]]

,機構名稱,geometry
5,吉星診所,POINT (121.56355000000008 25.037370000000067)


In [32]:
addr_info = geocoder.arcgis(location="25.045440000000042, 121.51695000000007 ", method="reverse") # address to coordinate
addr_json = addr_info.json
addr_json

{'address': '江恩婦產科診所',
 'city': '中正區',
 'country': 'TWN',
 'lat': 25.04544,
 'lng': 121.51695,
 'ok': True,
 'postal': '100',
 'raw': {'address': {'Match_addr': '江恩婦產科診所',
   'LongLabel': '台北市中正區黎明里許昌街8號江恩婦產科診所, 100',
   'ShortLabel': '江恩婦產科診所',
   'Addr_type': 'POI',
   'Type': 'Medical Clinic',
   'PlaceName': '江恩婦產科診所',
   'AddNum': '8',
   'Address': '許昌街8號',
   'Block': '',
   'Sector': '',
   'Neighborhood': '黎明里',
   'District': '',
   'City': '中正區',
   'MetroArea': '',
   'Subregion': '',
   'Region': '台北市',
   'RegionAbbr': '',
   'Territory': '',
   'Postal': '100',
   'PostalExt': '',
   'CntryName': '台灣',
   'CountryCode': 'TWN'},
  'location': {'x': 121.51695,
   'y': 25.04544,
   'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}},
 'region': '台北市',
 'state': '台北市',
 'status': 'OK'}

In [36]:
village_border = gpd.read_file('taipei_village_border', encoding='utf-8')
village_border.head(2)

,AREA,NEW,FULL,PERF_ID,COUN_ID,CPID,CPTID,CPTVID,NPID,NPTID,...,TM2_MIN_X,TM2_MIN_Y,MAX_X,MAX_Y,MIN_X,MIN_Y,SECT_NAME,LIE_NAME,sn,geometry
0,1.632654e+07,6.301200e+09,臺北市北投區湖田里,63012.0,6301200,63012,6301200,6301200042,63012.0,6301200.0,...,302319.034,2783056.23,121.57361,25.21014,121.51897,25.15509,北投區,湖田里,1,"POLYGON ((302666.543 2785226.842, 302675.820 2..."
1,1.171811e+07,6.301100e+09,臺北市士林區菁山里,63011.0,6301100,63011,6301100,6301100047,63011.0,6301100.0,...,305593.829,2779658.04,121.58405,25.19538,121.55131,25.12429,士林區,菁山里,2,"POLYGON ((307802.170 2787372.759, 307831.907 2..."


In [37]:
village_border.crs = 'EPSG:3826'
village_border.crs

<Projected CRS: EPSG:3826>
Name: TWD97 / TM2 zone 121
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Taiwan, Republic of China - between 120°E and 122°E, onshore and offshore - Taiwan Island.
- bounds: (119.99, 20.41, 122.06, 26.72)
Coordinate Operation:
- name: Taiwan 2-degree TM zone 121
- method: Transverse Mercator
Datum: Taiwan Datum 1997
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [38]:
village_border = village_border.to_crs('EPSG:4326')
village_border.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [41]:
village_border.head(3)

,FULL,SECT_NAME,LIE_NAME,geometry
0,臺北市北投區湖田里,北投區,湖田里,"POLYGON ((121.52250 25.17467, 121.52259 25.174..."
1,臺北市士林區菁山里,士林區,菁山里,"POLYGON ((121.57354 25.19386, 121.57383 25.194..."
2,臺北市北投區大屯里,北投區,大屯里,"POLYGON ((121.51904 25.16951, 121.51930 25.169..."


In [40]:
village_border = village_border[["FULL", "SECT_NAME", "LIE_NAME", "geometry"]]

In [42]:
village_border.to_file(
    'village_border.geojson',
    driver='GeoJSON'
)